# Deploy and predict with Keras model on Cloud AI Platform.

**Learning Objectives**

1. Setup up the environment
1. Deploy trained Keras model to Cloud AI Platform
1. Online predict from model on Cloud AI Platform
1. Batch predict from model on Cloud AI Platform


## Introduction 
**Verify that you have previously Trained your Keras model. If not, go back to [train_keras_ai_platform_babyweight.ipynb](../solutions/train_keras_ai_platform_babyweight.ipynb) create them.**
In this notebook, we'll be deploying our Keras model to Cloud AI Platform and creating predictions.

We will set up the environment, deploy a trained Keras model to Cloud AI Platform, online predict from deployed model on Cloud AI Platform, and batch predict from deployed model on Cloud AI Platform.

Each learning objective will correspond to a __#TODO__ in this student lab notebook -- try to complete this notebook first and then review the [solution notebook](../solutions/deploy_keras_ai_platform_babyweight.ipynb).

## Set up environment variables and load necessary libraries

Import necessary libraries.

In [1]:
import os

### Lab Task #1: Set environment variables.

Set environment variables so that we can use them throughout the entire lab. We will be using our project name for our bucket, so you only need to change your project and region.

In [2]:
%%bash
PROJECT=$(gcloud config list project --format "value(core.project)")
echo "Your current GCP Project Name is: "$PROJECT

Your current GCP Project Name is: qwiklabs-gcp-04-6c7883a7efcc


In [3]:
# Change these to try this notebook out
PROJECT = "qwiklabs-gcp-04-6c7883a7efcc"  # TODO 1: Replace with your PROJECT
BUCKET = "qwiklabs-gcp-04-6c7883a7efcc"  # defaults to PROJECT
REGION = "us-central1"  # TODO 1: Replace with your REGION

In [4]:
os.environ["BUCKET"] = BUCKET
os.environ["REGION"] = REGION
os.environ["TFVERSION"] = "2.1"

In [5]:
%%bash
gcloud config set compute/region $REGION
gcloud config set ai_platform/region global

Updated property [compute/region].
Updated property [ai_platform/region].


## Check our trained model files

Let's check the directory structure of our outputs of our trained model in folder we exported the model to in our last [lab](../solutions/10_train_keras_ai_platform_babyweight.ipynb). We'll want to deploy the saved_model.pb within the timestamped directory as well as the variable values in the variables folder. Therefore, we need the path of the timestamped directory so that everything within it can be found by Cloud AI Platform's model deployment service.

In [8]:
%%bash
gsutil ls gs://cloud-training-demos/babyweight/trained_model

gs://cloud-training-demos/babyweight/trained_model/checkpoint
gs://cloud-training-demos/babyweight/trained_model/events.out.tfevents.1529347276.cmle-training-master-a137ac0fff-0-9q8r4
gs://cloud-training-demos/babyweight/trained_model/graph.pbtxt
gs://cloud-training-demos/babyweight/trained_model/model.ckpt-342784.data-00000-of-00003
gs://cloud-training-demos/babyweight/trained_model/model.ckpt-342784.data-00001-of-00003
gs://cloud-training-demos/babyweight/trained_model/model.ckpt-342784.data-00002-of-00003
gs://cloud-training-demos/babyweight/trained_model/model.ckpt-342784.index
gs://cloud-training-demos/babyweight/trained_model/model.ckpt-342784.meta
gs://cloud-training-demos/babyweight/trained_model/model.ckpt-376661.data-00000-of-00003
gs://cloud-training-demos/babyweight/trained_model/model.ckpt-376661.data-00001-of-00003
gs://cloud-training-demos/babyweight/trained_model/model.ckpt-376661.data-00002-of-00003
gs://cloud-training-demos/babyweight/trained_model/model.ckpt-376661.i

In [ ]:
# copy trained canonical model

In [18]:
%%bash
gsutil -m cp -R gs://cloud-training-demos/babyweight/trained_model gs://${BUCKET}/babyweight/

Copying gs://cloud-training-demos/babyweight/trained_model/checkpoint...
Copying gs://cloud-training-demos/babyweight/trained_model/eval/events.out.tfevents.1529348264.cmle-training-master-a137ac0fff-0-9q8r4...
Copying gs://cloud-training-demos/babyweight/trained_model/events.out.tfevents.1529347276.cmle-training-master-a137ac0fff-0-9q8r4...
Copying gs://cloud-training-demos/babyweight/trained_model/export/exporter/1529355466/saved_model.pb...
Copying gs://cloud-training-demos/babyweight/trained_model/export/exporter/1529355466/variables/variables.data-00000-of-00001...
Copying gs://cloud-training-demos/babyweight/trained_model/graph.pbtxt...       
Copying gs://cloud-training-demos/babyweight/trained_model/export/exporter/1529355466/variables/variables.index...
Copying gs://cloud-training-demos/babyweight/trained_model/model.ckpt-342784.data-00000-of-00003...
Copying gs://cloud-training-demos/babyweight/trained_model/model.ckpt-342784.data-00001-of-00003...
Copying gs://cloud-training

In [19]:
%%bash
gsutil ls gs://${BUCKET}/babyweight/trained_model

gs://qwiklabs-gcp-04-6c7883a7efcc/babyweight/trained_model/checkpoint
gs://qwiklabs-gcp-04-6c7883a7efcc/babyweight/trained_model/events.out.tfevents.1529347276.cmle-training-master-a137ac0fff-0-9q8r4
gs://qwiklabs-gcp-04-6c7883a7efcc/babyweight/trained_model/graph.pbtxt
gs://qwiklabs-gcp-04-6c7883a7efcc/babyweight/trained_model/model.ckpt-342784.data-00000-of-00003
gs://qwiklabs-gcp-04-6c7883a7efcc/babyweight/trained_model/model.ckpt-342784.data-00001-of-00003
gs://qwiklabs-gcp-04-6c7883a7efcc/babyweight/trained_model/model.ckpt-342784.data-00002-of-00003
gs://qwiklabs-gcp-04-6c7883a7efcc/babyweight/trained_model/model.ckpt-342784.index
gs://qwiklabs-gcp-04-6c7883a7efcc/babyweight/trained_model/model.ckpt-342784.meta
gs://qwiklabs-gcp-04-6c7883a7efcc/babyweight/trained_model/model.ckpt-376661.data-00000-of-00003
gs://qwiklabs-gcp-04-6c7883a7efcc/babyweight/trained_model/model.ckpt-376661.data-00001-of-00003
gs://qwiklabs-gcp-04-6c7883a7efcc/babyweight/trained_model/model.ckpt-376661.da

In [28]:
%%bash
gsutil ls gs://${BUCKET}/babyweight/trained_model/export/exporter/1529355466/

gs://qwiklabs-gcp-04-6c7883a7efcc/babyweight/trained_model/export/exporter/1529355466/saved_model.pb
gs://qwiklabs-gcp-04-6c7883a7efcc/babyweight/trained_model/export/exporter/1529355466/variables/


In [29]:
%%bash
MODEL_LOCATION=$(gsutil ls -ld -- gs://${BUCKET}/babyweight/trained_model/export/exporter/1529355466/ \
                 | tail -1)
gsutil ls ${MODEL_LOCATION}

gs://qwiklabs-gcp-04-6c7883a7efcc/babyweight/trained_model/export/exporter/1529355466/saved_model.pb
gs://qwiklabs-gcp-04-6c7883a7efcc/babyweight/trained_model/export/exporter/1529355466/variables/


## Lab Task #2: Deploy trained model.

Deploying the trained model to act as a REST web service is a simple gcloud call. Complete __#TODO__ by providing location of saved_model.pb file to Cloud AI Platform model deployment service. The deployment will take a few minutes.

In [37]:
%%bash
MODEL_NAME="babyweight"
MODEL_VERSION="ml_on_gcp"
MODEL_LOCATION=gs://${BUCKET}/babyweight/trained_model/export/exporter/1529355466/
echo "Deleting and deploying $MODEL_NAME $MODEL_VERSION from $MODEL_LOCATION"
gcloud ai-platform versions delete ${MODEL_VERSION} --model ${MODEL_NAME}
gcloud ai-platform models delete ${MODEL_NAME}

Deleting and deploying babyweight ml_on_gcp from gs://qwiklabs-gcp-04-6c7883a7efcc/babyweight/trained_model/export/exporter/


Using endpoint [https://ml.googleapis.com/]
This will delete version [ml_on_gcp]...

Do you want to continue (Y/n)?  Please enter 'y' or 'n':  
ERROR: (gcloud.ai-platform.versions.delete) NOT_FOUND: Field: name Error: The specified model version was not found.
- '@type': type.googleapis.com/google.rpc.BadRequest
  fieldViolations:
  - description: The specified model version was not found.
    field: name


CalledProcessError: Command 'b'MODEL_NAME="babyweight"\nMODEL_VERSION="ml_on_gcp"\nMODEL_LOCATION=gs://${BUCKET}/babyweight/trained_model/export/exporter/\necho "Deleting and deploying $MODEL_NAME $MODEL_VERSION from $MODEL_LOCATION"\ngcloud ai-platform versions delete ${MODEL_VERSION} --model ${MODEL_NAME}\ngcloud ai-platform models delete ${MODEL_NAME}\n'' returned non-zero exit status 1.

In [53]:
%%bash
MODEL_NAME="babyweight1234"
MODEL_VERSION="ml_on_gcp1234"
MODEL_LOCATION=gs://${BUCKET}/babyweight/trained_model/export/exporter/1529355466/
echo "Deleting and deploying $MODEL_NAME $MODEL_VERSION from $MODEL_LOCATION"
# gcloud ai-platform versions delete ${MODEL_VERSION} --model ${MODEL_NAME}
# gcloud ai-platform models delete ${MODEL_NAME}
gcloud ai-platform models create ${MODEL_NAME} --regions ${REGION}
gcloud ai-platform versions create ${MODEL_VERSION} \
    --model=${MODEL_NAME} \
    --origin=${MODEL_LOCATION} \
    --runtime-version=2.1 \
    --python-version=3.7

Deleting and deploying babyweight1234 ml_on_gcp1234 from gs://qwiklabs-gcp-04-6c7883a7efcc/babyweight/trained_model/export/exporter/1529355466/


Using endpoint [https://ml.googleapis.com/]
Created ai platform model [projects/qwiklabs-gcp-04-6c7883a7efcc/models/babyweight1234].
Using endpoint [https://ml.googleapis.com/]
Creating version (this might take a few minutes)......
...........................................................................................................................................................................................................................................................................done.


## Lab Task #3: Use model to make online prediction.

Complete __#TODO__s for both the Python and gcloud Shell API methods of calling our deployed model on Cloud AI Platform for online prediction.

### Python API

We can use the Python API to send a JSON request to the endpoint of the service to make it predict a baby's weight. The order of the responses are the order of the instances.

In [54]:
from oauth2client.client import GoogleCredentials
import requests
import json

MODEL_NAME = "babyweight1234"
MODEL_VERSION = "ml_on_gcp1234"

token = GoogleCredentials.get_application_default().get_access_token().access_token
api = "https://ml.googleapis.com/v1/projects/{}/models/{}/versions/{}:predict" \
         .format(PROJECT, MODEL_NAME, MODEL_VERSION)
headers = {"Authorization": "Bearer " + token}
data = {
  "instances": [
    {
      "is_male": "True",
      "mother_age": 26.0,
      "plurality": "Single(1)",
      "gestation_weeks": 39
    },
    {
      "is_male": "False",
      "mother_age": 29.0,
      "plurality": "Single(1)",
      "gestation_weeks": 38
    },
    {
      "is_male": "True",
      "mother_age": 26.0,
      "plurality": "Triplets(3)",
      "gestation_weeks": 39
    },
    {
      "is_male": "False",
      "mother_age": 33.0,
      "plurality": "Single(1)",
      "gestation_weeks": 39
    }
  ]
}
response = requests.post(api, json=data, headers=headers)
print(response.content)

b'{"error": "Prediction failed: Error during model execution: <_MultiThreadedRendezvous of RPC that terminated with:\\n\\tstatus = StatusCode.INVALID_ARGUMENT\\n\\tdetails = \\"input size does not match signature: 4!=5 len({gestation_weeks,is_male,mother_age,plurality}) != len({gestation_weeks,is_male,key,mother_age,plurality}). Sent extra: {}. Missing but required: {key}.\\"\\n\\tdebug_error_string = \\"{\\"created\\":\\"@1653772901.978912072\\",\\"description\\":\\"Error received from peer ipv4:127.0.0.1:8081\\",\\"file\\":\\"src/core/lib/surface/call.cc\\",\\"file_line\\":903,\\"grpc_message\\":\\"input size does not match signature: 4!=5 len({gestation_weeks,is_male,mother_age,plurality}) != len({gestation_weeks,is_male,key,mother_age,plurality}). Sent extra: {}. Missing but required: {key}.\\",\\"grpc_status\\":3}\\"\\n>"}'


The predictions for the four instances were: 5.33, 6.09, 2.50, and 5.86 pounds respectively when I ran it (your results might be different).

### gcloud shell API

Instead we could use the gcloud shell API. Create a newline delimited JSON file with one instance per line and submit using gcloud.

In [55]:
%%writefile inputs.json
{"is_male": "True", "mother_age": 26.0, "plurality": "Single(1)", "gestation_weeks": 39}
{"is_male": "False", "mother_age": 26.0, "plurality": "Single(1)", "gestation_weeks": 39}

Writing inputs.json


Now call `gcloud ai-platform predict` using the JSON we just created and point to our deployed `model` and `version`.

In [57]:
%%bash
gcloud ai-platform predict \
    --model=babyweight1234 \
    --json-instances=inputs.json \
    --version=ml_on_gcp1234

{
  "error": "Prediction failed: Error during model execution: <_MultiThreadedRendezvous of RPC that terminated with:\n\tstatus = StatusCode.INVALID_ARGUMENT\n\tdetails = \"input size does not match signature: 4!=5 len({gestation_weeks,is_male,mother_age,plurality}) != len({gestation_weeks,is_male,key,mother_age,plurality}). Sent extra: {}. Missing but required: {key}.\"\n\tdebug_error_string = \"{\"created\":\"@1653773583.465868896\",\"description\":\"Error received from peer ipv4:127.0.0.1:8081\",\"file\":\"src/core/lib/surface/call.cc\",\"file_line\":903,\"grpc_message\":\"input size does not match signature: 4!=5 len({gestation_weeks,is_male,mother_age,plurality}) != len({gestation_weeks,is_male,key,mother_age,plurality}). Sent extra: {}. Missing but required: {key}.\",\"grpc_status\":3}\"\n>"
}


Using endpoint [https://ml.googleapis.com/]


## Lab Task #4: Use model to make batch prediction.

Batch prediction is commonly used when you have thousands to millions of predictions. It will create an actual Cloud AI Platform job for prediction. Complete __#TODO__s so we can call our deployed model on Cloud AI Platform for batch prediction.

In [58]:
%%bash
INPUT=gs://${BUCKET}/babyweight/batchpred/inputs.json
OUTPUT=gs://${BUCKET}/babyweight/batchpred/outputs
gsutil cp inputs.json $INPUT
gsutil -m rm -rf $OUTPUT 
gcloud ai-platform jobs submit prediction babypred_$(date -u +%y%m%d_%H%M%S) \
    --data-format=TEXT \
    --region ${REGION} \
    --input-paths=$INPUT \
    --output-path=$OUTPUT \
    --model=babyweight1234 \
    --version=ml_on_gcp1234

jobId: babypred_220528_213515
state: QUEUED


Copying file://inputs.json [Content-Type=application/json]...
/ [1 files][  179.0 B/  179.0 B]                                                
Operation completed over 1 objects/179.0 B.                                      
CommandException: 1 files/objects could not be removed.
Job [babypred_220528_213515] submitted successfully.
Your job is still active. You may view the status of your job with the command

  $ gcloud ai-platform jobs describe babypred_220528_213515

or continue streaming the logs with the command

  $ gcloud ai-platform jobs stream-logs babypred_220528_213515


In [60]:
%%bash
gcloud ai-platform jobs describe babypred_220528_213515

createTime: '2022-05-28T21:35:30Z'
endTime: '2022-05-28T21:36:45Z'
errorMessage: Internal error occurred. Please retry in a few minutes. If you still
  experience errors, contact Cloud ML.
etag: -Xihm8qcjrs=
jobId: babypred_220528_213515
jobPosition: '0'
predictionInput:
  dataFormat: JSON
  inputPaths:
  - gs://qwiklabs-gcp-04-6c7883a7efcc/babyweight/batchpred/inputs.json
  outputPath: gs://qwiklabs-gcp-04-6c7883a7efcc/babyweight/batchpred/outputs
  region: us-central1
  runtimeVersion: '2.1'
  versionName: projects/qwiklabs-gcp-04-6c7883a7efcc/models/babyweight1234/versions/ml_on_gcp1234
predictionOutput:
  outputPath: gs://qwiklabs-gcp-04-6c7883a7efcc/babyweight/batchpred/outputs
startTime: '2022-05-28T21:35:31Z'
state: FAILED



View job in the Cloud Console at:
https://console.cloud.google.com/mlengine/jobs/babypred_220528_213515?project=qwiklabs-gcp-04-6c7883a7efcc

View logs at:
https://console.cloud.google.com/logs?resource=ml_job%2Fjob_id%2Fbabypred_220528_213515&project=qwiklabs-gcp-04-6c7883a7efcc


## Lab Summary:
In this lab, we set up the environment, deployed a trained Keras model to Cloud AI Platform, online predicted from deployed model on Cloud AI Platform, and batch predicted from deployed model on Cloud AI Platform.

Copyright 2019 Google Inc. Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0 Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License